$|J| \quad |I|$

$n_i \quad f_i \quad c_{ij}$

In [1]:
with open('UFLP-1.txt', 'r') as file:
    # Leer la primera línea (n = ubicaciones potenciales, m = puntos de demanda)
    n, m = map(int, file.readline().split())

    fixed_costs = []
    assignment_costs = []

    # Leer las siguientes líneas con los costos fijos y de asignación
    for _ in range(n):
        data = list(map(int, file.readline().split()))
        fixed_costs.append(data[1])  # Segundo valor es el costo fijo
        assignment_costs.append(data[2:])  # Los siguientes son los costos de asignación


In [2]:
n

50

In [3]:
m

100

In [7]:
fixed_costs[0]

4751

In [6]:
assignment_costs[0]

[707,
 164,
 75,
 771,
 276,
 81,
 460,
 376,
 900,
 47,
 700,
 664,
 271,
 158,
 623,
 522,
 953,
 105,
 914,
 927,
 726,
 600,
 294,
 33,
 113,
 176,
 349,
 427,
 420,
 828,
 124,
 778,
 147,
 471,
 62,
 583,
 108,
 665,
 384,
 253,
 101,
 603,
 515,
 232,
 476,
 803,
 629,
 388,
 441,
 492,
 929,
 381,
 457,
 143,
 741,
 234,
 558,
 136,
 17,
 823,
 938,
 10,
 351,
 513,
 662,
 272,
 87,
 636,
 829,
 242,
 628,
 62,
 40,
 894,
 524,
 552,
 514,
 157,
 449,
 135,
 692,
 663,
 645,
 206,
 676,
 389,
 942,
 588,
 345,
 180,
 930,
 253,
 495,
 606,
 401,
 939,
 839,
 139,
 626,
 369]

In [ ]:
import numpy as np
import numpy as np

def initialize_uflp(n, m, fixed_costs, assignment_costs):
    selected_facilities = set()  # Conjunto de instalaciones seleccionadas
    allocations = np.full(m, -1)  # Almacena a qué instalación se asigna cada punto de demanda
    return selected_facilities, allocations

def add_heuristic(n, m, fixed_costs, assignment_costs):
    """Implementa la heurística ADD para obtener una solución inicial."""
    selected_facilities, allocations = initialize_uflp(n, m, fixed_costs, assignment_costs)

    # Paso 1: Seleccionar la primera instalación
    costs = np.sum(assignment_costs, axis=1) + fixed_costs
    j_star = np.argmin(costs)
    selected_facilities.add(j_star)
    allocations[:] = j_star  # Asignar todos los puntos de demanda a esta instalación
    total_cost = costs[j_star]

    # Paso 2: Agregar instalaciones mientras haya reducción en el costo
    while True:
        g = np.zeros(n)
        for j in range(n):
            if j in selected_facilities:
                g[j] = np.inf  # No considerar instalaciones ya seleccionadas
            else:
                g[j] = fixed_costs[j] + sum(
                    min(assignment_costs[i, j] - assignment_costs[i, allocations[i]], 0)
                    for i in range(m)
                )
        
        j_star = np.argmin(g)
        if g[j_star] >= 0:
            break  # Si no hay mejora, detenerse

        selected_facilities.add(j_star)
        total_cost += g[j_star]

        # Reasignar puntos de demanda si es más barato con la nueva instalación
        for i in range(m):
            if assignment_costs[i, j_star] < assignment_costs[i, allocations[i]]:
                allocations[i] = j_star

    return selected_facilities, allocations, total_cost

def local_search(n, m, fixed_costs, assignment_costs, selected_facilities, allocations):
    """Búsqueda local para mejorar la solución."""
    improved = True
    best_cost = calculate_cost(fixed_costs, assignment_costs, selected_facilities, allocations)
    
    while improved:
        improved = False
        current_facilities = list(selected_facilities)

        # Intentar eliminar una instalación
        for j in current_facilities:
            if len(selected_facilities) == 1:
                break  # No podemos eliminar la última instalación
            
            new_facilities = selected_facilities - {j}
            new_allocations = np.full(m, -1)

            # Asignar cada punto de demanda a la mejor instalación disponible
            for i in range(m):
                best_j = min(new_facilities, key=lambda x: assignment_costs[i, x])
                new_allocations[i] = best_j

            new_cost = calculate_cost(fixed_costs, assignment_costs, new_facilities, new_allocations)
            if new_cost < best_cost:
                selected_facilities = new_facilities
                allocations = new_allocations
                best_cost = new_cost
                improved = True
                break  # Aplicar el primer mejoramiento encontrado

        # Intentar agregar una nueva instalación si no hubo mejora con eliminación
        if not improved:
            for j in range(n):
                if j in selected_facilities:
                    continue  # No considerar instalaciones ya seleccionadas

                new_facilities = selected_facilities | {j}
                new_allocations = np.full(m, -1)

                # Reasignar cada punto de demanda a la mejor instalación disponible
                for i in range(m):
                    best_j = min(new_facilities, key=lambda x: assignment_costs[i, x])
                    new_allocations[i] = best_j

                new_cost = calculate_cost(fixed_costs, assignment_costs, new_facilities, new_allocations)
                if new_cost < best_cost:
                    selected_facilities = new_facilities
                    allocations = new_allocations
                    best_cost = new_cost
                    improved = True
                    break

    return selected_facilities, allocations, best_cost

def calculate_cost(fixed_costs, assignment_costs, facilities, allocations):
    """Calcula el costo total de la solución dada."""
    fixed_cost = sum(fixed_costs[j] for j in facilities)
    assignment_cost = sum(assignment_costs[i, allocations[i]] for i in range(len(allocations)))
    return fixed_cost + assignment_cost

# Example usage:
selected_facilities, allocations, initial_cost = add_heuristic(n, m, fixed_costs, assignment_costs)
selected_facilities, allocations, improved_cost = local_search(n, m, fixed_costs, assignment_costs, selected_facilities, allocations)
print(f"Initial cost: {initial_cost}, Improved cost: {improved_cost}")


TypeError: list indices must be integers or slices, not tuple

In [ ]:
def read_uflp_instance(file_path):
    with open(file_path, 'r') as file:
        # Leer la primera línea (n = ubicaciones potenciales, m = puntos de demanda)
        n, m = map(int, file.readline().split())

        fixed_costs = []
        assignment_costs = []

        # Leer las siguientes líneas con los costos fijos y de asignación
        for _ in range(n):
            data = list(map(int, file.readline().split()))
            fixed_costs.append(data[1])  # Segundo valor es el costo fijo
            assignment_costs.append(data[2:])  # Los siguientes son los costos de asignación

    return n, m, fixed_costs, assignment_costs

# 

In [12]:
# Leer datos del archivo
file_path = "UFLP-1.txt"  # Reemplaza con la ruta correcta si es necesario
n, m, fixed_costs, assignment_costs = read_uflp_instance(file_path)

# Crear el modelo y ejecutar la heurística
uflp = UFLP(n, m, fixed_costs, assignment_costs)

print("Costo inicial (heurística ADD):", uflp.add_heuristic())
print("Costo mejorado (búsqueda local):", uflp.local_search())
print("Ubicaciones seleccionadas:", uflp.selected_facilities)
print("Asignaciones de demanda:", uflp.allocations)


ValueError: operands could not be broadcast together with shapes (100,) (50,) 

In [9]:


uflp = UFLP(n, m, fixed_costs, assignment_costs)



In [10]:
print("Costo inicial (heurística ADD):", uflp.add_heuristic())


ValueError: operands could not be broadcast together with shapes (100,) (50,) 